### Resources

Website: https://public.opendatasoft.com/explore/dataset/airbnb-listings/api/?disjunctive.host_verifications&disjunctive.amenities&disjunctive.features&rows=10000&refine.city=Berlin

API documentation: https://help.opendatasoft.com/apis/ods-search-v1/#search-api-v1


### Import libraries

In [1]:
import pandas as pd
import requests
import json
import matplotlib
from currency_converter import CurrencyConverter # for currency conversion
from datetime import date # to set date of ciurrency conversion

%matplotlib inline

### Define API

In [2]:
# get only listings in Berlin, Germany bei adding "&refine.city=Berlin"
url = "https://public.opendatasoft.com/api/records/1.0/search/?dataset=airbnb-listings&rows=10000&facet=host_verifications&facet=city&facet=country&facet=property_type&facet=room_type&facet=bed_type&refine.city=Berlin"
headers = {"accept": "application/json"}

In [3]:
r = requests.get(url, headers=headers)

In [4]:
r

<Response [200]>

In [5]:
# r.text - not running this command to increase performance of file

In [6]:
# r.json() - not running this command to increase performance of file

### Checking types

In [7]:
type(r.json())

dict

In [8]:
r.json().keys()

dict_keys(['nhits', 'parameters', 'records', 'facet_groups'])

In [9]:
type(r.json()["nhits"])

int

In [10]:
type(r.json()["parameters"])

dict

In [11]:
type(r.json()["records"])

list

In [12]:
type(r.json()["facet_groups"])

list

### Checking list types to convert these into dataframes (in case they include the data needed)

In [13]:
parameters = pd.DataFrame(r.json()["parameters"])

In [14]:
parameters

,dataset,timezone,rows,format,facet
0,airbnb-listings,UTC,10000,json,host_verifications
1,airbnb-listings,UTC,10000,json,city
2,airbnb-listings,UTC,10000,json,country
3,airbnb-listings,UTC,10000,json,property_type
4,airbnb-listings,UTC,10000,json,room_type
5,airbnb-listings,UTC,10000,json,bed_type


In [15]:
records = pd.DataFrame(r.json()["records"])

In [16]:
records.head()

,datasetid,recordid,fields,geometry,record_timestamp
0,airbnb-listings,bb8be9bb68e3707afd34330681bc98528d1b2e6c,"{'geolocation': [52.48054210735575, 13.4358354...","{'type': 'Point', 'coordinates': [13.435835487...",2017-07-12T09:10:34.946000+00:00
1,airbnb-listings,409485a0eb562591220cfaba82557224cfed3564,"{'review_scores_accuracy': 10, 'reviews_per_mo...","{'type': 'Point', 'coordinates': [13.439348091...",2017-07-12T09:10:34.946000+00:00
2,airbnb-listings,4bb6ddeb1668eae1c03aac92074d477fee7770e0,"{'review_scores_accuracy': 10, 'reviews_per_mo...","{'type': 'Point', 'coordinates': [13.447508780...",2017-07-12T09:10:34.946000+00:00
3,airbnb-listings,e7a39046b934f79c2b8cacfa1c3687dcbd8ea308,"{'review_scores_accuracy': 10, 'reviews_per_mo...","{'type': 'Point', 'coordinates': [13.447824889...",2017-07-12T09:10:34.946000+00:00
4,airbnb-listings,6cb1dad5bd38480231b953ab0076678305db633f,"{'review_scores_accuracy': 10, 'reviews_per_mo...","{'type': 'Point', 'coordinates': [13.441572631...",2017-07-12T09:10:34.946000+00:00


In [17]:
facet_groups = pd.DataFrame(r.json()["facet_groups"])

In [18]:
facet_groups

,facets,name
0,"[{'count': 20454, 'path': 'Berlin', 'state': '...",city
1,"[{'count': 19307, 'path': 'Real Bed', 'state':...",bed_type
2,"[{'count': 20454, 'path': 'Germany', 'state': ...",country
3,"[{'count': 20301, 'path': 'phone', 'state': 'd...",host_verifications
4,"[{'count': 18651, 'path': 'Apartment', 'state'...",property_type
5,"[{'count': 10224, 'path': 'Entire home/apt', '...",room_type


## Convert fields from DataFrame records into DataFrame itself

In [19]:
# records.fields - not running this command to increase performance of file

In [20]:
records_fields = records["fields"]

In [21]:
# list(records_fields) - not running this command to increase performance of file

In [22]:
pd.set_option('display.max_colwidth', -1)
#pd.DataFrame.from_dict(fields) 
fields = pd.DataFrame(list(records_fields))
#fields = fields.drop_duplicates() # remove duplicates
#fields.sort_values(by=['calendar_last_scraped'])
fields.head(2)

,geolocation,bathrooms,features,cancellation_policy,accommodates,calendar_last_scraped,neighbourhood_cleansed,street,calendar_updated,country_code,...,host_response_time,host_response_rate,cleaning_fee,review_scores_rating,house_rules,square_feet,interaction,monthly_price,weekly_price,notes
0,"[52.48054210735575, 13.435835487092247]",1.0,Host Has Profile Pic,flexible,3,2017-05-08,Rixdorf,"Berlin, Berlin 12043, Germany",16 months ago,DE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"[52.479951086588564, 13.439348091158159]",1.0,"Host Has Profile Pic,Host Identity Verified,Is Location Exact,Require Guest Profile Picture",moderate,2,2017-05-08,Rixdorf,"Neukölln, Berlin, Berlin 12043, Germany",2 weeks ago,DE,...,within an hour,100.0,20.0,100.0,"- Please, take off your shoes in the house. You can use the slippers.",NaN,NaN,NaN,NaN,NaN


### Dropping columns that are not needed

In [23]:
#List all columns
fields.columns

Index(['geolocation', 'bathrooms', 'features', 'cancellation_policy',
       'accommodates', 'calendar_last_scraped', 'neighbourhood_cleansed',
       'street', 'calendar_updated', 'country_code', 'property_type',
       'availability_365', 'id', 'market', 'city', 'availability_60',
       'picture_url', 'zipcode', 'host_location', 'availability_90', 'state',
       'amenities', 'host_listings_count', 'maximum_nights', 'listing_url',
       'latitude', 'last_scraped', 'host_since', 'bedrooms', 'description',
       'experiences_offered', 'price', 'host_picture_url', 'extra_people',
       'smart_location', 'neighbourhood_group_cleansed', 'host_verifications',
       'number_of_reviews', 'xl_picture_url', 'host_about', 'guests_included',
       'host_url', 'host_id', 'availability_30',
       'calculated_host_listings_count', 'name', 'medium_url', 'bed_type',
       'country', 'longitude', 'summary', 'beds', 'minimum_nights',
       'thumbnail_url', 'host_name', 'scrape_id', 'host_total

In [24]:
# Check NaNs to decide what to drop with regards to neighborhood
pd.set_option('display.max_rows', 100)
fields.isna().sum()

# keep price, neighbourhood_group_cleansed, id, cleaning_fee

geolocation                       0   
bathrooms                         39  
features                          10  
cancellation_policy               0   
accommodates                      0   
calendar_last_scraped             0   
neighbourhood_cleansed            0   
street                            0   
calendar_updated                  0   
country_code                      0   
property_type                     0   
availability_365                  0   
id                                0   
market                            36  
city                              0   
availability_60                   0   
picture_url                       19  
zipcode                           360 
host_location                     58  
availability_90                   0   
state                             27  
amenities                         63  
host_listings_count               6   
maximum_nights                    0   
listing_url                       0   
latitude                 

In [25]:
# drop columns

fields = (fields.drop(['geolocation', 'bathrooms', 'features', 'cancellation_policy',
       'accommodates', 'calendar_last_scraped', 'neighbourhood_cleansed',
       'street', 'calendar_updated', 'country_code', 'property_type',
       'availability_365', 'market', 'city', 'availability_60',
       'picture_url', 'zipcode', 'host_location', 'availability_90', 'state',
       'amenities', 'host_listings_count', 'maximum_nights', 'listing_url',
       'latitude', 'last_scraped', 'host_since', 'bedrooms', 'description',
       'experiences_offered', 'host_picture_url', 'extra_people',
       'smart_location', 'host_verifications',
       'number_of_reviews', 'xl_picture_url', 'host_about', 'guests_included',
       'host_url', 'host_id', 'availability_30',
       'calculated_host_listings_count', 'name', 'medium_url', 'bed_type',
       'country', 'longitude', 'summary', 'beds', 'minimum_nights',
       'thumbnail_url', 'host_name', 'scrape_id', 'host_total_listings_count',
       'host_thumbnail_url', 'room_type', 'review_scores_accuracy',
       'reviews_per_month', 'first_review', 'transit', 'host_neighbourhood',
       'review_scores_cleanliness', 'neighborhood_overview', 'space', 'access',
       'security_deposit', 'review_scores_value', 'review_scores_location',
       'review_scores_checkin', 'review_scores_communication', 'last_review',
       'neighbourhood', 'host_response_time', 'host_response_rate',
       'review_scores_rating', 'house_rules', 'square_feet',
       'interaction', 'monthly_price', 'weekly_price', 'notes'], axis=1))

In [26]:
fields.head()

,id,price,neighbourhood_group_cleansed,cleaning_fee
0,10103806,45.0,Neukölln,NaN
1,16187267,45.0,Neukölln,20.0
2,16247375,18.0,Neukölln,NaN
3,412418,50.0,Neukölln,20.0
4,6076066,65.0,Neukölln,15.0


In [27]:
#check if Nans exist
fields.isna().sum()

id                              0   
price                           5   
neighbourhood_group_cleansed    0   
cleaning_fee                    4176
dtype: int64

In [28]:
# drop lines where price = NaN
fields.dropna(subset=['price'], inplace=True)


In [29]:
# show lines where price = 0
fields[fields.price == 0]

,id,price,neighbourhood_group_cleansed,cleaning_fee
1241,18537794,0.0,Charlottenburg-Wilm.,NaN
5258,18581047,0.0,Mitte,11.0
7422,18606311,0.0,Friedrichshain-Kreuzberg,10.0


In [30]:
# drop lines where price = 0
fields = fields[fields.price != 0]

In [31]:
# check if lines with prive = 0 replaced
fields[fields.price == 0]

,id,price,neighbourhood_group_cleansed,cleaning_fee


In [32]:
# Replace NaN values in cleaning_fee with 0
fields = fields.fillna(0)

In [33]:
# check if values with NaN were replaced:
fields.isna().sum()

id                              0
price                           0
neighbourhood_group_cleansed    0
cleaning_fee                    0
dtype: int64

In [34]:
#convert price from USD to EUR, using currency converter, rate as of 2020-01-08
c = CurrencyConverter()
fields['price'] = fields['price'].div(c.convert(1, 'EUR', 'USD', date=date(2020, 1, 8))) 
fields['price']=fields['price'].astype(int)


fields['cleaning_fee'] = fields['cleaning_fee'].div(c.convert(1, 'EUR', 'USD', date=date(2020, 1, 8))) 
fields['cleaning_fee']=fields['cleaning_fee'].astype(int)
fields[['price','cleaning_fee']].head(10)


,price,cleaning_fee
0,40,0
1,40,17
2,16,0
3,44,17
4,58,13
5,35,26
6,17,0
7,21,0
8,35,0
9,23,0


In [35]:
fields.describe()

# check for outliers 

,price,cleaning_fee
count,9992.000000,9992.000000
mean,51.089572,13.946357
std,43.049383,17.722777
min,0.000000,0.000000
25%,26.000000,0.000000
50%,40.000000,8.000000
75%,62.000000,22.000000
max,895.000000,256.000000


In [36]:
fields['price'].value_counts()

44     758
35     736
26     712
31     519
22     501
      ... 
235    1  
323    1  
331    1  
92     1  
0      1  
Name: price, Length: 200, dtype: int64

In [37]:
# drop row with price 0
fields = fields[fields.price != 0]

In [38]:
fields['price'].value_counts()

44     758
35     736
26     712
31     519
22     501
      ... 
235    1  
323    1  
331    1  
395    1  
895    1  
Name: price, Length: 199, dtype: int64

In [39]:
fields.describe()
# take those value for comparison to hotel prices

,price,cleaning_fee
count,9991.000000,9991.000000
mean,51.094685,13.947753
std,43.048503,17.723114
min,8.000000,0.000000
25%,26.000000,0.000000
50%,40.000000,8.000000
75%,62.000000,22.000000
max,895.000000,256.000000


In [40]:
# rename columns to match columns in other data sources
fields.rename(columns={'id': 'name', 'neighbourhood_group_cleansed': 'area', 'price': 'price'}, inplace=True)

In [41]:
fields.head()

,name,price,area,cleaning_fee
0,10103806,40,Neukölln,0
1,16187267,40,Neukölln,17
2,16247375,16,Neukölln,0
3,412418,44,Neukölln,17
4,6076066,58,Neukölln,13


In [42]:
# drop column cleaning fee to have same format as other sources
fields = fields.drop(['cleaning_fee'], axis =1)

# add column to indicate where these data come from as this is needed 
# to run analysis in merged file later and
# fill column source with value "Airbnb"
fields['source'] = "airbnb" 

## 

fields.head()

,name,price,area,source
0,10103806,40,Neukölln,airbnb
1,16187267,40,Neukölln,airbnb
2,16247375,16,Neukölln,airbnb
3,412418,44,Neukölln,airbnb
4,6076066,58,Neukölln,airbnb


### Reset Index

In [43]:
fields = fields.reset_index(drop=True)
fields.head()

,name,price,area,source
0,10103806,40,Neukölln,airbnb
1,16187267,40,Neukölln,airbnb
2,16247375,16,Neukölln,airbnb
3,412418,44,Neukölln,airbnb
4,6076066,58,Neukölln,airbnb


## Export *.pkl

In [44]:
fields.to_pickle('2017_airbnb_api_data_clean.pkl')

## Export *.csv

In [45]:
fields.to_csv('2017_airbnb_api_data_clean.csv', sep=',', index=False)